In [67]:
from docopt import docopt
import pandas as pd
import os
import tempfile
import shutil
from glob import glob
import sys
import subprocess as proc

import datman as dm
import datman.utils

In [68]:
output_dir = '/scratch/smansour/CARTBIND/combined/dmriprep'
dtifit_dir = '/scratch/mjoseph/test/dtifit'
QCdir = os.path.join(dtifit_dir, 'QC')
tmpdirbase = os.path.join(QCdir,'tmp')
QC_bet_dir = os.path.join(QCdir,'BET')
QC_V1_dir = os.path.join(QCdir, 'directions')
QC_FM_dir = os.path.join(QCdir, 'FM')
QC_Mag_dir = os.path.join(QCdir, 'Mag')
QC_res_dir = os.path.join(QCdir, 'res')
QC_SH_dir = os.path.join(QCdir, 'SH')

dir_list = [tmpdirbase, QC_bet_dir, QC_V1_dir, QC_res_dir, QC_SH_dir]

In [69]:
for qc_dir in dir_list:
    if not os.path.exists(qc_dir):
        os.makedirs(qc_dir)

In [70]:
grad_out = os.path.join(tmpdirbase,'ramp.gif')
create_gradient_file(grad_out,'red-yellow')

In [71]:
maskpics = []
V1pics = []
Respics = []
SHpics = []

In [72]:
allFAmaps = glob('{}/sub-*/ses-*/dwi/dtifit__FA*'.format(output_dir))

In [ ]:
for FAmap in allFAmaps:
    subid = FAmap.split('/')[-4]
    sesid = FAmap.split('/')[-3]
    tmpdir = os.path.join(tmpdirbase, subid, sesid)
    if not os.path.exists(tmpdir):
        os.makedirs(tmpdir)
    basename = '{}_{}_'.format(subid, sesid)
    pathbase = FAmap.replace('dtifit__FA.nii.gz','')
    pathdir = os.path.dirname(pathbase)
    SHtmp = os.path.join(tmpdir,'SHmap.gif')
    SHpic = os.path.join(QC_SH_dir,basename + 'SH.gif')
    SHpics.append(SHpic)
    SH_overlay(os.path.join(pathdir,'eddy_corrected_noise.nii.gz'), SHtmp, grad_out)
    if os.path.exists(os.path.join(pathdir,'eddy_corrected_noise.nii.gz')):
        gif_gridtoline(SHtmp,SHpic)

In [81]:
for FAmap in allFAmaps:
    subid = FAmap.split('/')[-4]
    sesid = FAmap.split('/')[-3]
    tmpdir = os.path.join(tmpdirbase, subid, sesid)
    if not os.path.exists(tmpdir):
        os.makedirs(tmpdir)
    basename = '{}_{}_'.format(subid, sesid)
    pathbase = FAmap.replace('dtifit__FA.nii.gz','')
    pathdir = os.path.dirname(pathbase)

    maskpic = os.path.join(QC_bet_dir,basename + 'b0_bet_mask.gif')
    maskpics.append(maskpic)
    mask_overlay(os.path.join(pathdir,'{}_{}_dwi_denoised_unr_avg_b0_brain.nii.gz'.format(subid, sesid)), 
                 os.path.join(pathdir,'{}_{}_dwi_denoised_unr_avg_b0_brain_mask.nii.gz'.format(subid, sesid)), maskpic)
    V1pic = os.path.join(QC_V1_dir,basename + 'dtifit_V1.gif')
    V1pics.append(V1pic)
    V1_overlay(FAmap,pathbase + 'dtifit__V1.nii.gz', V1pic)

#    SHtmp = os.path.join(tmpdir,'SHmap.gif')
#    SHpic = os.path.join(QC_SH_dir,basename + 'SH.gif')
#    SHpics.append(SHpic)
#    SH_overlay(os.path.join(pathdir,'eddy_corrected_noise.nii.gz'), SHtmp, grad_out)
#    gif_gridtoline(SHtmp,SHpic)

run(slices /scratch/smansour/CARTBIND/combined/dmriprep/sub-034/ses-01/dwi/sub-034_ses-01_dwi_denoised_unr_avg_b0_brain.nii.gz /scratch/smansour/CARTBIND/combined/dmriprep/sub-034/ses-01/dwi/sub-034_ses-01_dwi_denoised_unr_avg_b0_brain_mask.nii.gz -o /scratch/mjoseph/test/dtifit/QC/tmp/sub-034/ses-01/B0masked.gif) failed with returncode 1. STDERR: b''
run(convert /scratch/mjoseph/test/dtifit/QC/tmp/sub-034/ses-01/B0masked.gif -resize 384x384 /scratch/mjoseph/test/dtifit/QC/tmp/sub-034/ses-01/B0masked.gif) failed with returncode 1. STDERR: b"convert: unable to open image `/scratch/mjoseph/test/dtifit/QC/tmp/sub-034/ses-01/B0masked.gif': No such file or directory @ error/blob.c/OpenBlob/2712.\nconvert: no images defined `/scratch/mjoseph/test/dtifit/QC/tmp/sub-034/ses-01/B0masked.gif' @ error/convert.c/ConvertImageCommand/3210.\n"
run(convert /scratch/mjoseph/test/dtifit/QC/tmp/sub-034/ses-01/B0masked.gif -crop 100x33%+0+0 /scratch/mjoseph/test/dtifit/QC/tmp/sub-034/ses-01/sag.gif) faile

In [76]:
qchtml = open(os.path.join(QCdir,'qc_BET.html'),'w')
qchtml.write('<HTML><TITLE>DTIFIT BET QC page</TITLE>')
qchtml.write('<BODY BGCOLOR=#333333>\n')
qchtml.write('<h1><font color="white">DTIFIT BET QC page</font></h1>')
for pic in maskpics:
    relpath = os.path.relpath(pic,QCdir)
    qchtml.write('<a href="'+ relpath + '" style="color: #99CCFF" >')
    qchtml.write('<img src="' + relpath + '" "WIDTH=800" > ')
    qchtml.write(relpath + '</a><br>\n')
qchtml.write('</BODY></HTML>\n')
qchtml.close() # you can omit in most cases as the destructor will call it

## write an html page that shows all the V1 pics
qchtml = open(os.path.join(QCdir,'qc_directions.html'),'w')
qchtml.write('<HTML><TITLE>DTIFIT directions QC page</TITLE>')
qchtml.write('<BODY BGCOLOR=#333333>\n')
qchtml.write('<h1><font color="white">DTIFIT directions QC page</font></h1>')
for pic in V1pics:
    relpath = os.path.relpath(pic,QCdir)
    qchtml.write('<a href="'+ relpath + '" style="color: #99CCFF" >')
    qchtml.write('<img src="' + relpath + '" "WIDTH=800" > ')
    qchtml.write(relpath + '</a><br>\n')
qchtml.write('</BODY></HTML>\n')
qchtml.close() # you can omit in most cases as the destructor will call it

# write an html page that shows all the SH residual pics
qchtml = open(os.path.join(QCdir,'qc_SH.html'),'w')
qchtml.write('<HTML><TITLE>SH residual QC page</TITLE>')
qchtml.write('<BODY BGCOLOR=#333333>\n')
qchtml.write('<h1><font color="white">SH residual QC page</font></h1>')
for pic in SHpics:
    relpath = os.path.relpath(pic,QCdir)
    qchtml.write('<a href="'+ relpath + '" style="color: #99CCFF" >')
    qchtml.write('<img src="' + relpath + '" "WIDTH=800" > ')
    qchtml.write(relpath + '</a><br>\n')
qchtml.write('</BODY></HTML>\n')
qchtml.close() # you can omit in most cases as the destructor will call it

## write an html page that shows all the Res pics
qchtml = open(os.path.join(QCdir,'qc_res.html'),'w')
qchtml.write('<HTML><TITLE>DTIFIT residual QC page</TITLE>')
qchtml.write('<BODY BGCOLOR=#333333>\n')
qchtml.write('<h1><font color="white">DTIFIT residual QC page</font></h1>')
for pic in Respics:
    relpath = os.path.relpath(pic,QCdir)
    qchtml.write('<a href="'+ relpath + '" style="color: #99CCFF" >')
    qchtml.write('<img src="' + relpath + '" "WIDTH=800" > ')
    qchtml.write(relpath + '</a><br>\n')
qchtml.write('</BODY></HTML>\n')
qchtml.close() # you can omit in most cases as the destructor will call it

In [41]:
def gif_gridtoline(input_gif,output_gif):
    '''
    uses imagemagick to take a grid from fsl slices and convert to one line (like in slicesdir)
    '''
    dm.utils.run(['convert',input_gif, '-resize', '384x384',input_gif])
    dm.utils.run(['convert', input_gif,\
        '-crop', '100x33%+0+0', os.path.join(tmpdir,'sag.gif')])
    dm.utils.run(['convert', input_gif,\
        '-crop', '100x33%+0+128', os.path.join(tmpdir,'cor.gif')])
    dm.utils.run(['convert', input_gif,\
        '-crop', '100x33%+0+256', os.path.join(tmpdir,'ax.gif')])
    dm.utils.run(['montage', '-mode', 'concatenate', '-tile', '3x1', \
        os.path.join(tmpdir,'sag.gif'),\
        os.path.join(tmpdir,'cor.gif'),\
        os.path.join(tmpdir,'ax.gif'),\
        os.path.join(output_gif)])

In [74]:
def mask_overlay(background_nii,mask_nii, overlay_gif):
    '''
    use slices from fsl to overlay the mask on the background (both nii)
    then make the grid to a line for easier scrolling during QC
    '''
    dm.utils.run(['slices', background_nii, mask_nii, '-o', os.path.join(tmpdir,'B0masked.gif')])
    gif_gridtoline(os.path.join(tmpdir,'B0masked.gif'),overlay_gif)

In [13]:
def V1_overlay(background_nii,V1_nii, overlay_gif):
    '''
    use fslsplit to split the V1 image and take pictures of each direction
    use slices from fsl to get the background and V1 picks (both nii)
    recolor the V1 image using imagemagick
    then make the grid to a line for easier scrolling during QC
    '''
    dm.utils.run(['slices',background_nii,'-o',os.path.join(tmpdir,"background.gif")])
    dm.utils.run(['fslmaths',background_nii,'-thr','0.15','-bin',os.path.join(tmpdir,'FAmask.nii.gz')])
    dm.utils.run(['fslsplit', V1_nii, os.path.join(tmpdir,"V1")])
    for axis in ['0000','0001','0002']:
        dm.utils.run(['fslmaths',os.path.join(tmpdir,'V1'+axis+'.nii.gz'), '-abs', \
            '-mul', os.path.join(tmpdir,'FAmask.nii.gz'), os.path.join(tmpdir,'V1'+axis+'abs.nii.gz')])
        dm.utils.run(['slices',os.path.join(tmpdir,'V1'+axis+'abs.nii.gz'),'-o',os.path.join(tmpdir,'V1'+axis+'abs.gif')])
        # docmd(['convert', os.path.join(tmpdir,'V1'+axis+'abs.gif'),\
        #         '-fuzz', '15%', '-transparent', 'black', os.path.join(tmpdir,'V1'+axis+'set.gif')])
    dm.utils.run(['convert', os.path.join(tmpdir,'V10000abs.gif'),\
        os.path.join(tmpdir,'V10001abs.gif'), os.path.join(tmpdir,'V10002abs.gif'),\
        '-set', 'colorspace', 'RGB', '-combine', '-set', 'colorspace', 'sRGB',\
        os.path.join(tmpdir,'dirmap.gif')])
    gif_gridtoline(os.path.join(tmpdir,'dirmap.gif'),overlay_gif)

In [14]:
def SSE_overlay(sse,out,grad):
    '''
    Arguments:
        sse                        Full path to SSE file
        out                        Full path to output
        grad                       Full path to gradient look-up map

    Steps:
    1. Clever/Hacky thresholding so maximum intensity is 2
    2. Generate slices
    3. Use gradient map to color greyscale image
    4. Background filling with 0 fuzziness to prevent leakage
    '''
    slice_out = out.replace('.nii.gz','.gif')
    cmd1 = 'fslmaths {} -sub 3 -mul -1 -thr 0 -mul -1 -add 3 {}'.format(sse,out)
    cmd2 = 'slices {} -o {}'.format(out, slice_out)
    cmd3 = 'convert {} {} -clut {}'.format(slice_out, grad, slice_out)
    cmd4 = 'convert {} -fill black -draw "color 0,0 floodfill" {}'.format(slice_out,slice_out)
    cmdlist = [cmd1, cmd2, cmd3, cmd4]
    outputs = [call(c) for c in cmdlist]
    return

In [58]:
def SH_overlay(SH,out,grad):
    '''
    Arguments:
        SH                         Full path to SSE file
        out                        Full path to output
        grad                       Full path to gradient look-up map

    Steps:
    1. Clever/Hacky thresholding so maximum intensity is 2
    2. Generate slices
    3. Use gradient map to color greyscale image
    4. Background filling with 0 fuzziness to prevent leakage
    '''
    slice_out = out.replace('.nii.gz','.gif')
    cmd1 = 'fslmaths {} -sub 0.3 -mul -1 -thr 0 -mul -1 -add 0.3 {}'.format(SH,out)
    cmd2 = 'slices {} -o {}'.format(out, slice_out)
    cmd3 = 'convert {} {} -clut {}'.format(slice_out, grad, slice_out)
    cmd4 = 'convert {} -fill black -draw "color 0,0 floodfill" {}'.format(slice_out,slice_out)
    cmdlist = [cmd1, cmd2, cmd3, cmd4]
    outputs = [call(c) for c in cmdlist]
    return

In [24]:
def create_gradient_file(output,color):
    '''
    Arguments:
        output                    Full path to output file
        color                     String argument of Image-Magick 'color:color'
    '''

    cmd = 'convert -size 10x20 gradient:{} {}'.format(color,output)
    call(cmd)
    return

In [18]:
def call(cmd):
    p = proc.Popen(cmd,shell=True,stdin=proc.PIPE, stderr=proc.PIPE)
    std, err = p.communicate()

    if p.returncode:
        print('{} failed with error {}'.format(cmd,err))
    return

In [16]:
def get_sse(sub):
    sse = '{}_dtifit_sse.nii.gz'.format(sub)
    return os.path.join(dtifitdir,sub,sse)